In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import pandas as pd
import numpy as np

# Đọc dữ liệu từ file CSV
data = pd.read_csv('A.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Chỉ sử dụng cột 'Close' để dự đoán
close_prices = data['Close'].values.reshape(-1, 1)

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Chuẩn hóa dữ liệu
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_close_prices = scaler.fit_transform(close_prices)

# Tạo chuỗi thời gian
look_back = 60
train_size = int(len(scaled_close_prices) * 0.8)
test_size = len(scaled_close_prices) - train_size
train, test = scaled_close_prices[0:train_size,:], scaled_close_prices[train_size-look_back:]

train_generator = TimeseriesGenerator(train, train, length=look_back, batch_size=20)
test_generator = TimeseriesGenerator(test, test, length=look_back, batch_size=1)

In [3]:

# Xây dựng mô hình Bi-LSTM
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Huấn luyện mô hình
model.fit(train_generator, epochs=100, verbose=1)

# Đánh giá mô hình
prediction = model.predict(test_generator)
predicted_prices = scaler.inverse_transform(prediction)

# So sánh kết quả dự đoán với dữ liệu thực tế
actual_prices = scaler.inverse_transform(test[look_back:])
print(predicted_prices)
print(actual_prices)

Epoch 1/100
202/202 [==============================] - 9s 29ms/step - loss: 0.0035
Epoch 2/100
202/202 [==============================] - 7s 33ms/step - loss: 0.0012
Epoch 3/100
202/202 [==============================] - 8s 38ms/step - loss: 0.0028
Epoch 4/100
202/202 [==============================] - 8s 41ms/step - loss: 0.0013
Epoch 5/100
202/202 [==============================] - 9s 43ms/step - loss: 6.8963e-04
Epoch 6/100
202/202 [==============================] - 8s 39ms/step - loss: 6.6311e-04
Epoch 7/100
202/202 [==============================] - 9s 42ms/step - loss: 5.0528e-04
Epoch 8/100
202/202 [==============================] - 10s 50ms/step - loss: 6.6356e-04
Epoch 9/100
202/202 [==============================] - 9s 43ms/step - loss: 5.1861e-04
Epoch 10/100
202/202 [==============================] - 7s 33ms/step - loss: 4.0746e-04
Epoch 11/100
202/202 [==============================] - 6s 32ms/step - loss: 3.4312e-04
Epoch 12/100
202/202 [==============================] - 